In [4]:
import requests
from pprint import pprint
import pandas as pd
from datetime import datetime

In [5]:
base_url = "https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/"
response  = requests.get(base_url + "GetCaseList?states=1,51&fromYear=2018&toYear=2021&minNumOfVehicles=0&maxNumOfVehicles=10&format=json")
data = response.json()

In [6]:
df = pd.DataFrame(data["Results"][0])
df

,CountyName,CrashDate,Fatals,Peds,Persons,St_Case,State,StateName,TotalVehicles
0,TALLADEGA (121),/Date(1515150000000-0500)/,1,0,1,10001,1,Alabama,2
1,WALKER (127),/Date(1515390480000-0500)/,2,0,2,10002,1,Alabama,1
2,CHILTON (21),/Date(1515469800000-0500)/,1,0,2,10003,1,Alabama,2
3,BALDWIN (3),/Date(1515520920000-0500)/,1,0,2,10004,1,Alabama,1
4,JEFFERSON (73),/Date(1516363740000-0500)/,1,0,2,10005,1,Alabama,2
...,...,...,...,...,...,...,...,...,...
4995,CLARKE (43),/Date(1598622780000-0400)/,1,0,2,510474,51,Virginia,2
4996,NEW KENT (127),/Date(1598635800000-0400)/,1,0,1,510475,51,Virginia,1
4997,NORTHUMBERLAND (133),/Date(1598395500000-0400)/,1,0,1,510476,51,Virginia,1
4998,BEDFORD (19),/Date(1598688300000-0400)/,1,0,1,510477,51,Virginia,1


In [7]:
#filter out the problematic dates
df_filtered = df[df["CrashDate"] != "/Date(-62135578800000-0500)/"]
df_filtered["CrashDate"] = df_filtered["CrashDate"].apply(lambda x: datetime.fromtimestamp(int(x[6:19])/1000))

df_filtered.sort_values(by = "CrashDate")


C:\Users\david\AppData\Local\Temp\ipykernel_16272\568120754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["CrashDate"] = df_filtered["CrashDate"].apply(lambda x: datetime.fromtimestamp(int(x[6:19])/1000))


,CountyName,CrashDate,Fatals,Peds,Persons,St_Case,State,StateName,TotalVehicles
12,MADISON (89),2018-01-01 19:05:00,2,0,4,10013,1,Alabama,3
11,LIMESTONE (83),2018-01-01 21:47:00,1,0,1,10012,1,Alabama,1
14,ST. CLAIR (115),2018-01-02 13:30:00,1,0,2,10015,1,Alabama,1
13,HALE (65),2018-01-02 20:05:00,1,0,2,10014,1,Alabama,2
15,CULLMAN (43),2018-01-03 15:45:00,1,0,2,10016,1,Alabama,1
...,...,...,...,...,...,...,...,...,...
3321,ETOWAH (55),2021-12-31 09:59:00,1,0,2,10884,1,Alabama,2
3320,MOBILE (97),2021-12-31 11:30:00,1,0,3,10883,1,Alabama,1
4473,NEW KENT (127),2021-12-31 14:30:00,1,0,3,510858,51,Virginia,1
3324,MONTGOMERY (101),2021-12-31 20:39:00,1,0,1,10887,1,Alabama,1


In [8]:
us_states_dict = {
    'Alabama': 1,
    'Alaska': 2,
    'Arizona': 3,
    'Arkansas': 4,
    'California': 5,
    'Colorado': 6,
    'Connecticut': 7,
    'Delaware': 8,
    'Florida': 9,
    'Georgia': 10,
    'Hawaii': 11,
    'Idaho': 12,
    'Illinois': 13,
    'Indiana': 14,
    'Iowa': 15,
    'Kansas': 16,
    'Kentucky': 17,
    'Louisiana': 18,
    'Maine': 19,
    'Maryland': 20,
    'Massachusetts': 21,
    'Michigan': 22,
    'Minnesota': 23,
    'Mississippi': 24,
    'Missouri': 25,
    'Montana': 26,
    'Nebraska': 27,
    'Nevada': 28,
    'New Hampshire': 29,
    'New Jersey': 30,
    'New Mexico': 31,
    'New York': 32,
    'North Carolina': 33,
    'North Dakota': 34,
    'Ohio': 35,
    'Oklahoma': 36,
    'Oregon': 37,
    'Pennsylvania': 38,
    'Rhode Island': 39,
    'South Carolina': 40,
    'South Dakota': 41,
    'Tennessee': 42,
    'Texas': 43,
    'Utah': 44,
    'Vermont': 45,
    'Virginia': 46,
    'Washington': 47,
    'West Virginia': 48,
    'Wisconsin': 49,
    'Wyoming': 50
}


In [9]:
for idx, row in df_filtered.iterrows():
    state_case = row["St_Case"]
    state_case_year = row["CrashDate"].strftime('%Y')
    state_number = us_states_dict[row["StateName"]]

    #Get the specific details for each crash
    response = requests.get(base_url + f"GetCaseDetails?stateCase={state_case}&caseYear={state_case_year}&state={state_number}&format=json")
    data = response.json()

KeyboardInterrupt: 